In [ ]:
from __future__ import print_function, division
from sympy import init_printing
from sympy.printing.codeprinter import CodePrinter
from sympy.printing.precedence import precedence
from sympy.printing.str import StrPrinter
from sympy import *
import ast
import numpy as np 

In [ ]:
#init_session()

In [ ]:
name = "free_energy_def.dat"
tmp = open(name, "r")
tmp=tmp.read()
exec(tmp)

In [ ]:
name = "ML_parameter.dat" #load n_conv, con_dim,kill (killsmall)
tmp = open(name, "r")
tmp=tmp.read()
exec(tmp)
kill=float(kill)
print(n_parameter,conv_dim,n_conv,kill)

In [ ]:
name = "assign_symbols_final.dat"
tmp = open(name, "r")
tmp=tmp.read()
tmp=tmp.replace("eps0","epsilon0")
tmp=tmp.replace("sepsilon0","seps0")

exec(tmp)

In [ ]:
tmp

In [ ]:
fed_eqn

In [ ]:

name = "all_parameter.dat"
tmp = open(name, "r")
tmp=tmp.read()
#print(tmp)
tmp=tmp.replace("/varible:0",":")
tmp=tmp.replace("/const_1:0",":")

for i in range(100):
    tmp=tmp.replace("_"+str(i),"")
tmp=tmp.replace("[[","")
tmp=tmp.replace("]]","")
tmp=tmp.replace("\n",",")
tmp=tmp.replace(":","\":")
tmp=tmp.replace(",",",\"")
tmp="{\""+tmp+"}"
tmp=tmp.replace(",\"}","}")
#print(tmp)
dict_all=ast.literal_eval(tmp)

In [ ]:
for i in list(dict_all.keys()):
    if(np.abs(dict_all[i])<kill):
        dict_all[i]=0
    #dict_all[i]=np.round(dict_all[i],1)

In [ ]:
tmp = "{"
for i in list(dict_all.keys()):
    if(dict_all[i]==0):
        tmp+=str(i)+"->"+"0,"
tmp+="}"
#tmp[-2]=""
print(tmp)

In [ ]:
fed_eqn().subs(dict_all)

In [ ]:
data_file = '../LJ_data_all_correct/'
f=np.loadtxt(data_file+"MC_parameter.dat")
L = float(f[0])
N = int(f[2])
dx = (L/N)
print(N,L,dx)
w_array=np.loadtxt("w_array.dat")

In [ ]:
srho=symbols('rho') 
f=fed_eqn()
#f=f.subs({sn0:np.sum(w_array[0])*dx*srho})
#f=f.subs({sn1:np.sum(w_array[1])*dx*srho})
#f=f.subs({sn2:np.sum(w_array[2])*dx*srho})
#f=f.subs({sn3:np.sum(w_array[3])*dx*srho})
f=f.subs(dict_all)

In [ ]:
def non_zeros(a):
    a=float(a)
    power=np.floor(np.log10(np.abs(a)))
    coeff = np.round(np.abs(a)/10**power)
    sign = np.round(a/np.abs(a))
    return sign*coeff*10**(power)

In [ ]:
f2=f
for a in preorder_traversal(f):
    if isinstance(a, Float):
        f2 = f2.subs(a, non_zeros(a))
nsimplify(f2, tolerance=1e-4)

In [ ]:
f2=f
for a in preorder_traversal(f):
    if isinstance(a, Float):
        f2 = f2.subs(a, round(a, 1))

In [ ]:
nsimplify(f2, tolerance=1e-4)

In [ ]:
name = "fed_latex_simple.dat"
tmp_file = open(name, "w")
tmp_eqn=str(latex(nsimplify(f2, tolerance=1e-4)))
count=0
for _ in range (100):
    for i in range (len(tmp_eqn)):
        if(tmp_eqn[i:i+6]=="\left("):
            for j in range (i+5,len(tmp_eqn)):
                if(tmp_eqn[j:j+6]=="\left(" or tmp_eqn[j]=="y"):
                    break;
                if(tmp_eqn[j:j+6]=="right)"):
                    replace_str=tmp_eqn[i:j+6]
                    print("y_"+str(count)+"="+replace_str)
                    tmp_eqn=tmp_eqn.replace(tmp_eqn[i:j+6],
                                            "y_{"+str(count)+"}")
                    count+=1
                    break
        if(tmp_eqn[i:i+3]=="e^{"):
            left_par=1
            for j in range (i+3,len(tmp_eqn)):
                if(tmp_eqn[j]=="}"):
                    left_par-=1
                if(tmp_eqn[j]=="{"):
                    left_par+=1
                if(left_par==0):
                    replace_str=tmp_eqn[i+3:j]
                    print("y_"+str(count)+"="+replace_str)
                    tmp_eqn=tmp_eqn.replace(tmp_eqn[i+3:j],
                                            "y_{"+str(count)+"}")
                    count+=1
                    break
                if(tmp_eqn[j]=="y"):
                    break;
                            
print(tmp_eqn)
tmp_eqn=tmp_eqn.replace("log","ln")
tmp_file.write("&\\FML = \\,\\int dx \\, ")
tmp_file.write(tmp_eqn)
tmp_file.write("\\\\")
tmp_file.write("\n")
tmp_eqn=str(latex(nsimplify(f2, tolerance=1e-4)))
Count=0
for _ in range (100):
    for i in range (len(tmp_eqn)):
        if(tmp_eqn[i:i+6]=="\left("):
            for j in range (i+5,len(tmp_eqn)):
                if(tmp_eqn[j:j+6]=="\left("or tmp_eqn[j]=="y"):
                    break;
                
                if(tmp_eqn[j:j+6]=="right)"):
                    replace_str=tmp_eqn[i:j+6]
                    if(Count!=count-1):
                        tmp_file.write("y_"+str(Count)+"="+replace_str[6:-7]+",\\,")
                    elif(Count==count-1):
                        tmp_file.write("y_"+str(Count)+"="+replace_str[6:-7])
                    
                    tmp_eqn=tmp_eqn.replace(tmp_eqn[i:j+6],
                                            "y_{"+str(count)+"}")
                    Count+=1
                    break
        if(tmp_eqn[i:i+3]=="e^{"):
            left_par=1
            for j in range (i+3,len(tmp_eqn)):
                if(tmp_eqn[j]=="}"):
                    left_par-=1
                if(tmp_eqn[j]=="{"):
                    left_par+=1
                if(left_par==0):
                    replace_str=tmp_eqn[i+3:j]
                    if(Count!=count-1):
                        tmp_file.write("y_"+str(Count)+"="+replace_str+",\\,")
                    elif(Count==count-1):
                        tmp_file.write("y_"+str(Count)+"="+replace_str)
                    
                    tmp_eqn=tmp_eqn.replace(tmp_eqn[i+3:j],
                                            "y_{"+str(count)+"}")
                    Count+=1
                    break
                if(tmp_eqn[j]=="y"):
                    break;

tmp_file.close()
